In [5]:
import optuna
import torch
from torch.utils.data import DataLoader, TensorDataset
from deep_models import OptionsNN  # Assume your original file is named options_nn.py
import logging
from preprocess import preprocess
import pandas as pd


In [9]:
# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,  # Set the minimum logging level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Specify the format of log messages
    handlers=[
        logging.FileHandler("tuning.log"),  # Log to a file named tuning.log
        logging.StreamHandler()  # Also output logs to the console
    ]
)

# Define the objective function for optuna to minimize test loss
def objective(trial):
    # Suggest hyperparameters to tune
    input_size = 22  # Based on your fixed input features size
    eta = trial.suggest_loguniform('eta', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    init_type = trial.suggest_categorical('init_type', ['kaiming', 'xavier', None])
    epochs = trial.suggest_int('epochs', 10, 100)

    # Create the model with suggested hyperparameters
    model = OptionsNN(input_size=input_size, eta=eta, init_type=init_type)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    df = pd.read_csv("../data/options.csv")  # load data
    df_subsample = df.sample(frac=0.3, random_state=69)  # ~400,000
    X_train_scaled, y_train, X_test_scaled, y_test = preprocess(df_subsample, remove_outliers=True)
    X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
    X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)
    train_dataset = TensorDataset(X_train_scaled, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    train_size = len(train_dataset)

    for epoch in range(epochs):
        model.train()
        perm = torch.randperm(train_size).to(device)  # Shuffle training data
        for i in range(0, train_size, batch_size):
            idxs = perm[i:i + batch_size]
            x_bat, y_bat = X_train_scaled[idxs], y_train[idxs]
            x_bat, y_bat = x_bat.to(device), y_bat.to(device)

            model.optimizer.zero_grad()
            y_pred = model(x_bat)
            loss = model.criterion(y_pred, y_bat)
            loss.backward()
            model.optimizer.step()

    # Evaluation on the test set after training
    test_loss = model.evaluate(X_test_scaled, y_test)

    # Log the trial's result
    logging.info(f'Trial complete - Test loss: {test_loss:.4f}')

    # Return the test loss for optuna to minimize
    return test_loss


# Run the hyperparameter tuning process


In [10]:
def run_tuning():
    logging.info("Starting hyperparameter tuning to minimize test loss.")
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    # Print and log the best hyperparameters
    logging.info(f"Best hyperparameters: {study.best_params}")
    logging.info(f"Best test loss: {study.best_value}")


if __name__ == "__main__":
    run_tuning()


[I 2024-10-24 00:55:50,269] A new study created in memory with name: no-name-1270d063-3b86-420f-bb1f-09e83daba9ca
C:\Users\Artem Frenk\AppData\Local\Temp\ipykernel_19336\2609833236.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 1e-5, 1e-1)


Removing outliers using isolation forests
IsolationForest outlier detection removed 8.096906899118396% rows
871699 rows remaining
Splitting data into training and testing sets.
Standardizing training data.


[I 2024-10-24 00:59:40,756] Trial 0 finished with value: 54.66297912597656 and parameters: {'eta': 0.04183435731527055, 'batch_size': 64, 'init_type': 'xavier', 'epochs': 22}. Best is trial 0 with value: 54.66297912597656.
C:\Users\Artem Frenk\AppData\Local\Temp\ipykernel_19336\2609833236.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 1e-5, 1e-1)


Removing outliers using isolation forests
IsolationForest outlier detection removed 9.035865125704008% rows
862793 rows remaining
Splitting data into training and testing sets.
Standardizing training data.


[I 2024-10-24 01:20:30,794] Trial 1 finished with value: 37.08955764770508 and parameters: {'eta': 0.0003795926472466274, 'batch_size': 32, 'init_type': 'xavier', 'epochs': 66}. Best is trial 1 with value: 37.08955764770508.
C:\Users\Artem Frenk\AppData\Local\Temp\ipykernel_19336\2609833236.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 1e-5, 1e-1)


...

[I 2024-10-24 20:50:12,355] Trial 47 finished with value: 37.864933013916016 and parameters: {'eta': 0.0013595460820877633, 'batch_size': 32, 'init_type': 'kaiming', 'epochs': 44}. Best is trial 41 with value: 34.82684326171875.
C:\Users\Artem Frenk\AppData\Local\Temp\ipykernel_19336\2609833236.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 1e-5, 1e-1)


Removing outliers using isolation forests
IsolationForest outlier detection removed 9.094062401818453% rows
862241 rows remaining
Splitting data into training and testing sets.
Standardizing training data.


[I 2024-10-24 21:16:23,252] Trial 48 finished with value: 35.71623229980469 and parameters: {'eta': 0.0003930593898181984, 'batch_size': 32, 'init_type': 'kaiming', 'epochs': 83}. Best is trial 41 with value: 34.82684326171875.
C:\Users\Artem Frenk\AppData\Local\Temp\ipykernel_19336\2609833236.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 1e-5, 1e-1)


Removing outliers using isolation forests
IsolationForest outlier detection removed 7.182619256972603% rows
880371 rows remaining
Splitting data into training and testing sets.
Standardizing training data.


[I 2024-10-24 21:40:54,802] Trial 49 finished with value: 36.11970520019531 and parameters: {'eta': 0.0008945982753437412, 'batch_size': 32, 'init_type': 'kaiming', 'epochs': 74}. Best is trial 41 with value: 34.82684326171875.


array([[3.14676916e-02, 0.00000000e+00, 0.00000000e+00, ...,
        9.36688184e-02, 2.62968595e-02, 7.96907790e-02],
       [2.79712814e-02, 4.22953184e-04, 1.01781837e-03, ...,
        3.36519746e-02, 1.31369941e-02, 1.07238227e-01],
       [1.15835268e-02, 0.00000000e+00, 0.00000000e+00, ...,
        2.60951649e-02, 8.34655812e-03, 2.19255146e-02],
       ...,
       [2.64232525e-03, 3.46682938e-05, 4.80515112e-05, ...,
        6.03196251e-02, 3.05398592e-02, 8.53127257e-02],
       [1.63610644e-02, 7.86276903e-03, 6.11564688e-03, ...,
        1.57250339e-01, 4.59359500e-02, 2.08292342e-01],
       [5.73304508e-02, 1.59820834e-03, 2.08805658e-03, ...,
        1.33333164e-01, 3.99768631e-02, 1.12438520e-01]])